In [1]:
from contextlib import nullcontext
import selenium.webdriver as wb
import time
import numpy as np
import datetime
# 导入库

In [2]:
class Spider(object):

    def __init__(self, display=True) -> None:
        super().__init__()
        option = wb.ChromeOptions()
        # 忽略driver自身的警告|错误等输出
        option.add_experimental_option(
            'excludeSwitches', ['enable-logging'])
        # 禁用图片加载,加速爬取url等页面数据,图片数据待爬取url后由requests库实现
        # option.add_experimental_option(
        #     'prefs', {"profile.managed_default_content_settings.images": 2})
        if display == False:
            option.headless = True
            print('程序运行，设置为无窗口模式')
        option.binary_location = r"C:\Program Files\Google\Chrome\Application\chrome.exe"
        # 浏览器驱动器初始化，需指明路径与参数
        try:
            self.driver = wb.Chrome(
                r"C:\Users\Ludy\Desktop\workspace\assets\chromedriver.exe", options=option)
        except:
            self.driver = wb.Chrome(
                r"C:\Users\Ludy_Lab\Desktop\workspace\assets\chromedriver.exe", options=option)

    def isElementPresent(self, by, value):
        # 从selenium.common.exceptions模块导入NoSuchElementException异常类
        from selenium.common.exceptions import NoSuchElementException
        try:
            element = self.driver.find_element(by=by, value=value)
        except NoSuchElementException as e:
            # 打印异常信息
            # print(e)
            # 发生了NoSuchElementException异常，说明页面中未找到该元素，返回False
            return False
        else:
            # 没有发生异常，表示在页面中找到了该元素，返回True
            return True

    def login(self):
        # 获取这个地址下的网页
        self.driver.get('http://yjskq.xjtu.edu.cn')
        self.random_sleep(3)
        usrid_elm = self.driver.find_element_by_css_selector(
            '#form1 > input.username')
        usrid_elm.send_keys('********')
        pwd_elm = self.driver.find_element_by_css_selector(
            '#form1 > input.pwd')
        pwd_elm.send_keys('********')
        login_elm = self.driver.find_element_by_css_selector('#account_login')
        login_elm.click()
        self.random_sleep(2)
        print('登录成功')

    def get_recent_kq(self):
        sw_elm = self.driver.find_element_by_css_selector(
            '#app > div > div.header.flex-box.flex-between.bg-white.box-shadow.fixed.top > div.navbar > ul > li:nth-child(3)')
        sw_elm.click()
        self.random_sleep(2)
        row1col1_elm = self.driver.find_element_by_xpath(
            '//*[@id="app"]/div/div[2]/div/div[1]/div/div[2]/div[2]/div[1]/div[3]/table/tbody/tr[1]/td[2]/div')
        row1col2_elm = self.driver.find_element_by_xpath(
            '//*[@id="app"]/div/div[2]/div/div[1]/div/div[2]/div[2]/div[1]/div[3]/table/tbody/tr[1]/td[3]/div')
        row1col3_elm = self.driver.find_element_by_xpath(
            '//*[@id="app"]/div/div[2]/div/div[1]/div/div[2]/div[2]/div[1]/div[3]/table/tbody/tr[1]/td[4]/div')
        row1col4_elm = self.driver.find_element_by_xpath(
            '//*[@id="app"]/div/div[2]/div/div[1]/div/div[2]/div[2]/div[1]/div[3]/table/tbody/tr[1]/td[5]/div')
        row1col5_elm = self.driver.find_element_by_xpath(
            '//*[@id="app"]/div/div[2]/div/div[1]/div/div[2]/div[2]/div[1]/div[3]/table/tbody/tr[1]/td[6]/div')
        row1col6_elm = self.driver.find_element_by_xpath(
            '//*[@id="app"]/div/div[2]/div/div[1]/div/div[2]/div[2]/div[1]/div[3]/table/tbody/tr[1]/td[7]/div')
        # print(row1col1_elm.text, row1col2_elm.text, row1col3_elm.text,
        #       row1col4_elm.text, row1col5_elm.text, row1col6_elm.text)
        return row1col1_elm.text + "_" + row1col2_elm.text + '_' + row1col3_elm.text + \
            row1col4_elm.text + "_" + row1col5_elm.text + '_' + row1col6_elm.text

    def isNeedtoGotoStudy(self, current_time=-1,kb_list=[]):
        if current_time == -1:
            current_time = datetime.datetime.now()
        weekday = current_time.weekday()+1
        time_hour = current_time.hour
        time_minute = current_time.minute
        print(f'当前时间：{current_time.year}_{current_time.month}_{current_time.day}({weekday})_{current_time.hour}_{current_time.minute}')
        lesson_rank = self.hour2lesson(time_hour, time_minute)
        if lesson_rank == None:
            return False
        if kb_list == []:
            kb_list = self.get_kb()
        print('本周上课时间表为：')
        for i in range(len(kb_list)):
            for j in range(len(kb_list[i])):
                if kb_list[i][j]=='Null':
                    print('N',end=' ')
                else:
                    print('Y',end=' ')
            print('')
        if kb_list[weekday-1][lesson_rank-1] != 'Null':
            return True
        else:
            return False

    def isSummer(self):
        current_time = datetime.datetime.now()
        # day = current_time.day
        month = current_time.month
        if month >= 5 and month < 10:
            print('当前为夏季作息时间')
            return True
        else:
            print('当前为冬季作息时间')
            return False

    def hour2lesson(self, hour, minute):
        summer = False
        if self.isSummer():
            summer = True
        if hour == 8:
            return 1
        if hour == 9:
            return 2
        if hour == 10:
            return 3
        if hour == 11:
            return 4
        if hour == 14:
            if summer:
                if minute >= 30:
                    return 5
            else:
                return 5
        if hour == 15:
            if summer:
                if minute >= 30:
                    return 6
                else:
                    return 5
            else:
                return 6
        if hour == 16:
            if summer:
                if minute >= 30:
                    return 7
                else:
                    return 6
            else:
                return 7
        if hour == 17:
            if summer:
                if minute >= 30:
                    return 8
                else:
                    return 7
            else:
                return 8
        if hour == 18:
            if summer:
                if minute >= 30:
                    return None
                else:
                    return 8
        if hour == 19:
            if summer:
                if minute >= 30:
                    return 9
            else:
                return 9
        if hour == 20:
            if summer:
                if minute >= 30:
                    return 10
                else:
                    return 9
            else:
                return 10
        if hour == 21:
            if summer:
                if minute >= 30:
                    return 11
                else:
                    return 10
            else:
                return 11
        if hour == 22:
            if summer:
                if minute >= 30:
                    return None
                else:
                    return 11
        return None

    def get_kb(self) -> list:
        self.login()
        self.random_sleep(2)
        kb_elm = self.driver.find_element_by_xpath(
            '//*[@id="app"]/div/div[1]/div[2]/ul/li[2]')
        kb_elm.click()
        self.random_sleep(3)
        worker_tp = self
        kb = []
        tmp = []
        for weekday in range(1, 8):
            tmp = []
            for lesson in range(1, 12):
                key = 'Null'
                xpath_lesson = rf'//*[@id="app"]/div/div[2]/div/div/div/div[2]/div[2]/div/div/div[{2+weekday}]/ul/li[{1+lesson}]/div/div/div[1]'
                xpath_teacher = rf'//*[@id="app"]/div/div[2]/div/div/div/div[2]/div[2]/div/div/div[{2+weekday}]/ul/li[{1+lesson}]/div/div/div[2]'
                xpath_classroom = rf'//*[@id="app"]/div/div[2]/div/div/div/div[2]/div[2]/div/div/div[{2+weekday}]/ul/li[{1+lesson}]/div/div/div[3]'
                if worker_tp.isElementPresent(by='xpath', value=xpath_lesson):
                    key = worker_tp.driver.find_element_by_xpath(xpath_lesson).text + '_' +\
                        worker_tp.driver.find_element_by_xpath(xpath_teacher).text + '_' +\
                        worker_tp.driver.find_element_by_xpath(
                            xpath_classroom).text
                tmp.append(key)
            kb.append(tmp)
        # self.random_sleep(100)
        return kb

    def random_sleep(self, delay=3):
        # 用numpy的随机数
        tmp = np.random.random()
        # 使用time.sleep来让进程暂停
        time.sleep(delay*(tmp+0.5))

In [3]:
# def sendmail():
#     import subprocess
#     cmd = ''

In [4]:
worker = Spider(display=True)
print(worker.isNeedtoGotoStudy(current_time=datetime.datetime.fromtimestamp(1609893510)))
# if worker.isNeedtoGotoStudy():
#     sendmail()

当前时间：2021_1_6(3)_8_38
当前为冬季作息时间
登录成功
本周上课时间表为：
N N Y Y Y Y N N N N N 
N N N N N N Y Y N N N 
Y Y Y Y N N Y Y N N N 
N N N N N N N N N N N 
N N N N Y Y Y Y N N N 
N N N N N N N N N N N 
N N N N N N N N N N N 
True
